<a href="https://colab.research.google.com/github/l2yao/translator/blob/main/Translate_a_Public_Domain_Book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Translate a public domain book

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Translate_a_Public_Domain_Book.ipynb"><img src = "../images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

In this notebook, you will explore Gemini model as a translation tool, demonstrating how to prepare data, create effective prompts, and save results into a `.txt` file.

This approach significantly improves knowledge accessibility. Various sources provide open-source books. In this notebook, you will use [Project Gutenberg](https://www.gutenberg.org/) as a resource.


This platform provides access to a wide range of books available for free download in PDF, eBook, TXT formats and more.

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2" tqdm

In [ ]:
import google.generativeai as genai

from tqdm import tqdm
from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Data preparation

You will translate a well-known book written by Arthur Conan Doyle from Polish (trans. Eugenia Żmijewska) to English about the detective Sherlock Holmes. Here are the titles in Polish and English:



* Polish Title: *Tajemnica Baskerville'ów: dziwne przygody Sherlocka Holmes*

* English Title: *The Hound of the Baskervilles*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Translation/chengde/book1.txt") as f:
      book = f.read()

Books contain all sorts of fictional or historical descriptions, some of them rather literal and might cause the model to stop from performing translation query. To prevent some of those exceptions users are able to change `safety_setting` from default to more open approach.

Go ahead and initialize the Gemini 1.5 Flash model for this task.

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash-exp')

def generate_output(prompt):
    response = model.generate_content(
        prompt,
        request_options = {
            "timeout": 1000
        }
    )

    try:
        return response.text
    except Exception as ex:
        raise ex

Begin by counting how many tokens are in the book to know if you need to proceed with the next step (split text into smaller chunks).

## Token Information

LLMs are constrained by token limits. In order to stay within the limit of Gemini 1.5 Flash, it's important to ensure the following:

* Input token limit should not exceed: 1,048,576 tokens
* Output token limit should not exceed: 8,192 tokens

Since input chunks of post-translation text will serve as outputs, it's advisable to split them into smaller segments.

As you will see in the example, there are over 1000 small chunks. Calling API for the token count of each one would take a long time. Because of this estimate 1 token is equal to 4 characters will be used.

To account for the possible miscalculation of tokens and output potentially being longer than input each chunk will use up to 4*5000 characters. Remember to adjust the maximum to your needs.

For more details on tokens in Gemini models, refer to the documentation [here](https://ai.google.dev/gemini-api/docs/models/gemini
).

### Split text into chunks
Adjust the number of whitespace characters used for splitting based on the book you are working with.

Then, ensure to consider token limitations to receive complete responses from the model.


In [ ]:
chunks = book.split("\n\n")

num_chunks = len(chunks)
print(f"Number of chunks: {num_chunks}")

Number of chunks: 51


In [ ]:
estimated_token_counts = []

for chunk in chunks:
    estimated_token_count = len(chunk)
    estimated_token_counts.append(estimated_token_count)

# You can print number of estimated tokens in each non empty chunk to see what are you working with
print(estimated_token_counts)

[515, 0, 539, 1252, 7, 14978, 7, 15714, 10, 15085, 10, 15833, 14, 15678, 14, 17483, 18, 12821, 25, 12918, 16, 14508, 38, 12174, 30, 13744, 30, 14133, 25, 13462, 44, 14425, 26, 12875, 26, 13685, 26, 14926, 26, 13289, 0, 26, 14759, 26, 13874, 27, 14520, 27, 13250, 27, 13480]


# Identify the largest number of tokens in a chunk to check if any exceed your token limitations. In this case, the chunks are very small, so this should not be an issue.

If you encounter this problem, consider splitting the text into even smaller chunks.

In [ ]:
max(estimated_token_counts)

17483

### Group smaller chunks together
As observed above, certain chunks contain very few tokens. While the LLM can translate these sentences accurately, providing larger context can enhance its understanding of the content.

Let's combine smaller chunks into larger ones for improved results.

In [ ]:
def split_chunk(chunk, max_len=6000):
    result = []
    index = 0
    while index < len(chunk):
      start = index
      if index + max_len < len(chunk):
        end = chunk.rfind("。", index, index + max_len) + 1
      else:
        end = len(chunk)
      result.append(chunk[start:end])
      index = end

    return result

def chunk_splitting(chunks, max_len=6000):
    result = []
    for chunk in chunks:
      result.extend(split_chunk(chunk, max_len))
    return result

In [ ]:
# Combine short chunks into larger groups based on token counts.
def chunk_grouping(chunks, token_counts, max_len=6000):
    grouped_chunks = []
    current_group = ""
    current_token_sum = 0

    # Process each chunk and group them based on token limits
    for index in range(len(chunks)):
        chunk = chunks[index]
        count = token_counts[index]
        if token_counts[index] == 0:
            continue

        current_group += "\n\n" + chunk
        current_token_sum += count  # Count in 1 token for newlines
        # Add a new chunk if there is space available in the current group.
        if current_token_sum + 1 + count > max_len:
            grouped_chunks.append(current_group)
            current_group = ""
            current_token_sum = 0


    return grouped_chunks


grouped_chunks = chunk_grouping(chunks, estimated_token_counts)

final_chunks = chunk_splitting(grouped_chunks)

In [ ]:
for chunk in final_chunks:
    print(len(chunk))

5987
5972
5342
5973
5988
3764
5985
5934
3180
5982
5953
3912
5978
5968
3750
5978
5996
5527
5995
5985
863
5998
5994
955
5956
5974
2598
5994
5996
226
5986
5987
1805
5989
5992
2186
5931
5984
1576
5968
5974
2531
6000
5953
952
5975
5984
1756
5962
5983
3011
5997
5976
1346
5985
5980
2824
5992
5959
1953
5989
5967
2595
5999
5986
1296
5964
5998
1549


The last step of splitting is checking if the non-estimated token counts are acceptable.

In [ ]:
token_counts = []
for chunk in final_chunks:
  response = model.count_tokens(chunk,
                                request_options = {'timeout': 600})
  token_count = response.total_tokens
  token_counts.append(token_count)
token_counts

[3719,
 3561,
 3613,
 1716,
 3609,
 3638,
 3563,
 583,
 3612,
 3645,
 3487,
 156,
 3587,
 3615,
 3645,
 638,
 3534,
 3618,
 3638,
 516,
 3508,
 3639,
 3595,
 1806,
 3539,
 3703,
 2147,
 3631,
 3567,
 2217,
 3616,
 3589,
 3276,
 3692,
 3721,
 1670,
 3741,
 3684,
 2802,
 3760,
 3720,
 3233,
 3531,
 3668,
 2581,
 3698,
 3687,
 3395,
 3699,
 3729,
 2186,
 3764,
 3702,
 2781,
 3728,
 3678,
 3678,
 3619,
 3695,
 2498,
 3617,
 3571,
 3585,
 3688,
 3648,
 2923,
 3628,
 3613,
 3420,
 3634,
 3662,
 2445,
 3690,
 3680,
 2650]

Despite using estimates, this split minimizes the calls necessary to translate the book well, and most chunks are of similar token counts to each other.

## Translation

Your book is ready for translations! The selected book has been divided into 13 larger chunks.

### Prompt

Specify the original language of your book and the language you would like it to be translated into.


In [ ]:
def translations(text,
                 inputLanguage='Chinese',
                 targetLanguage='English'):

    translation_prompt = f"""
    As a professional book translator,
    translate the following book from {inputLanguage} into {targetLanguage}.

    Book to Translate:
    {text}

    """

    return translation_prompt

### Translate chunks

The translation time depends on the amount of text being processed. Our selected book was 318 KB and the entire translation took 20 minutes.

Please note that larger texts may take several hours to complete.

In [ ]:
results = []
index = 0
translation = None

for chunk in tqdm(final_chunks):
    try:
        translation = generate_output(translations(chunk))
        results.append(translation)
    except Exception as ex:
        print(f"\nChunk was not included in translation due to possible sensitive content. Text: {chunk}")

    index += 1

100%|██████████| 69/69 [35:43<00:00, 31.07s/it]


All done, now you can print the chunk before and after translation to see the results.

You can see, that translation effectively maintains the narrative's novel quality, proper grammar and sentence structure.

Next step will be joining chunks together and saving the translated book.

In [ ]:
Markdown(final_chunks[0])



﻿編者序
「望子成龍，望女成鳳」，是每位家長的期盼；「為天下得人」，是每位教育工作者的志願。如何才能成為自己的孩子、自己的學生，乃至身邊所有人生命中的貴人，用愛與智慧的心燈，照亮彼此的一生？
二０一三年七月，中國教師發展基金會、國家教師科研基金「十二五」規劃重點課題「中國學校法制教育行動研究」總課題組，在南京舉辦了「學習傳統文化 做有道德的教育者」校長培訓班，蔡禮旭老師受邀義務演講，主題是「做孩子一生的貴人」，一共六課時。因在南京的演講尚未圓滿，蔡老師返回馬來西亞中華文化教育中心後，每週六繼續錄製此套課程，計有十七課時。因而此主題演講共二十三課時。
蔡老師以一顆真誠無私的愛心，將自身深入中華五千年教育智慧的心得，以及十多年弘揚文化、培育人才的經驗，毫無保留地奉獻給大眾。此次課程，是繼《幸福人生講座》、《細講弟子規》之後，蔡老師的又一菁華演講，不但值得所有家長及教育工作者細細體味，也值得所有為員工的幸福和大眾的感動而奮鬥的企業家、團隊領導者參考。
鑒於此，敝講記組根據蔡老師演講文字稍加整理，以饗讀者。若有內容匯整不當、或文字訛誤之處，祈望諸位仁者不吝賜教。
  馬來西亞中華文化教育中心 講記圖書組
二０一四年七月一日


目錄
做孩子一生的貴人提綱
第一講 釋題
第二講 釋題
第三講 生命的意義
第四講 人類的苦難
第五講 建國君民，教學為先
第六講 建國君民，教學為先
第七講 師道尊嚴：一、教師的重要性
第八講 師道尊嚴：二、尊師方能重道；三、振興師道
第九講 師道尊嚴：三、振興師道
第十講 師道尊嚴：三、振興師道；教育工作者的典範──至聖先師：一、孔子治學
第十一講 教育工作者的典範──至聖先師：二、孔子為師之風範
第十二講 教育工作者的典範──至聖先師：二、孔子為師之風範
第十三講 自我教育的重要性：一、教育者首先受教育
第十四講 自我教育的重要性：一、教育者首先受教育；二、勇於改過；三、嚴以律己，寬以待人
第十五講 自我教育的重要性：三 嚴以律己，寬以待人
第十六講 自我教育的重要性：三、嚴以律己，寬以待人
第十七講 自我教育的重要性：三、嚴以律己，寬以待人
第十八講 自我教育的重要性：四、敦倫盡分，閑邪存誠
第十九講 自我教育的重要性：四、敦倫盡分，閑邪存誠
第二十講 自我教育的重要性：四、敦倫盡分，閑邪存誠
第二十一講 自我教育的重要性：四、敦倫盡分，閑邪存誠
第二十二講 自我教育的重要性：四、敦倫盡分，閑邪存誠
第二十三講 自我教育的重要性：四、敦倫盡分，閑邪存誠


做孩子一生的貴人提綱
壹 釋題
貳 生命的意義
參 人類的苦難
肆 建國君民 教學為先
伍 師道尊嚴
   一 教師的重要性
1. 不知禮義不可以行法 （《群書治要360》第218句）
2. 玉不琢不成器
3. 古之學者 比物醜類 （《禮記‧學記》）
4. 師哉師哉 童子之命也
5. 文化承傳
   二 尊師方能重道
1. 凡學之道 嚴師為難 （《禮記‧學記》）
2. 是故古之聖王未有不尊師也，尊師則不論貴賤貧富矣（《群書治要360》第250句）
3. 一分誠敬得一分利益 十分誠敬得十分利益 有狀元學生，沒有狀元老師
4. 孝親是尊師的基礎
5. 古今對比
   三 振興師道
1. 自重而後人重
2. 夫知為人子 （《群書治要360》第177句）
3. 知本分──傳道、授業、解惑
4. 安貧樂道 為國育才
5. 自我提升
陸 教育工作者的典範──至聖先師
   一 孔子治學
1. 述而不作 信而好古 （《論語‧述而第七》）
2. 子曰 吾十有五而志於學 （《論語‧為政第二》）
3. 十室之邑 （《論語‧公冶長第五》）
4. 子曰 不曰如之何者 （《論語‧衛靈公第十五》）
知之者不如好之者 （《論語‧雍也第六》）
5. 子曰 由 誨女知之乎 （《論語‧為政第二》）
6. 君子食無求飽 （《論語‧學而第一》）   士志於道 （《論語‧里仁第四》）
7. 德之不修 （《論語‧述而第七》）
8. 學而不思則罔 （《論語‧為政第二》）
9. 子曰 三人行必有我師焉 （《論語‧述而第七》）
10. 賜也 汝以予為多學而識之者 （《論語‧衛靈公第十五》）
   二 孔子為師之風範
1. 智慧
   1. 務本
   2. 看得深遠
   3. 禁於未發
   4. 因材施教
   5. 機會教育
   6. 相觀而善
2. 仁德
   1. 以身作則
   2. 慈悲心
   3. 有教無類
3. 勇氣
4. 行儀
柒 自我教育的重要性
   一 教育者首先受教育
1. 君子務本 本立而道生
2. 融得性情上偏私 便是一大學問 消得家庭內嫌隙 便是一大經綸
3. 起心動念不自欺
4. 放下對立、控制、佔有
   二 勇於改過
1. 恥心
2. 畏心
3. 勇心
   三 嚴以律己 寬以待人
1. 嚴以律己
   1. 不貪
   2. 不瞋
   3. 不痴
   4. 不慢
   5. 不疑
2. 寬以待人
   1. 設身處地
   2. 不計較
   3. 厚道
   四 敦倫盡份 閑邪存誠
     1.作之君 作之親 作之師
    （1）君：以身作則 使命感 大公無私 知人善任 納諫
    （2）親：養育 高度信任 不棄不捨 不求回報
    （3）師：道統承傳 培養民族棟梁 開人智慧
2.人臣本分
    （1）四種主動
    （2）務本
    （3）立節 盡忠 勸諫 舉賢
3.正確的思想觀念
    （1）對人
    （2）對事
    （3）對物


第一講 釋題


尊敬的葉總賢伉儷、莫祕書長、陳老師，諸位教育界的同道們，諸位長輩、兄弟姐妹們，大家早上好！
看到諸位校長，諸位教育界的同道，內心非常的喜悅。孔子說，「有朋自遠方來，不亦樂乎？」而我們講台上的背景圖，是油菜花田，數以千萬計的美麗的花朵，讓我們感受到諸位校長同道們，是為了孩子一生幸福的人生在用心，在著想。諸位校長的這份用心，一定會在孩子的人生當中開花結果，就像這片油菜花田一樣。在諸位校長背後，可能影響的是幾萬、幾十萬孩子的人生幸福。所以今天跟大家交流的題目，是「做孩子一生的貴人」。
而從昨天到了我們南京居美馨教育中心，一直都在感動當中。像昨天晚上大家自我介紹，其實有一句俗話講，「人生得一知己，死而無憾」，在聽著校長們的自我分享，我們可以感覺到，在這裡會遇到很多知音，會有一種惺惺相惜的感覺。而且又會體會到，經典告訴我們的，「德不孤，必有鄰」、「四海之內皆兄弟」。而且今天這個緣分，我們走在這個弘揚聖賢教育的路上，會形成一個教學的團隊。我們一百多個人，大家一起同心協力，以後有好的經驗、好的例子都可以互相分享，那個力量是非常非常大的。
我們以前一個人孤軍奮戰，現在結識到我們祖國大地所有的志士仁人，對教育非常有使命的校長們，大家共聚一堂，這是非常殊勝的緣分。所謂「知緣、惜緣（珍惜這個緣分）、造緣」，造我們往後所有學校老師、同仁、孩子們幸福人生的緣。「知恩、感恩、報恩」，我們感恩我們南京居美馨教育中心，對於整個聖賢教育的弘揚、整個教育的未來，他們有一份使命感，良苦用心，在暑假期間辦了這樣的課程。我們也感謝主辦單位，我們教育部中國教師發展基金會，國家教師科研基金，這個課題小組，跟我們居美馨中心一起合辦這樣的因緣，這背後有這些負責人非常多的良苦、非常多的用心。包含我們整個居美馨工作的團隊，我們每天要吃好、睡好，對生活種種的照顧，多虧我們居美馨團隊給我們的照顧。
所以我們知恩，知道背後這麼多人的付出，我們遇到工作人員，我們要露出燦爛的微笑，感謝他們的付出。不只在這裡，我們回到學校，感謝所有老師同仁，大家同心協力的付出，也感謝所有家長對學校的支持，我們都活在感恩的世界裡。
事實上，人只要心存感恩，他決定是一個幸福的人。那知恩、感恩，接著什麼？報恩。透過自己的提升，在學校的落實，能夠真正把倫理道德教育在自己的學校紮根、弘揚開，讓自己的學校成為我們中國落實倫理道德教育的示範學校，這樣叫光照神州大地。我們已經有榜樣了，像吉林松花江中學，他們一個王琦老師在自己的班上用心的做，不為名也不為利，就希望給孩子好的未來。大家想一想，一個人的真心能夠發揮多大的作用？可能很難想像。就像一顆種子落到土壤裡，它生根、發芽、茁壯了，它又結了很多的果實，這些果實裡面又有種子。可能一顆種子最後結的果實、結的種子，有上千上萬，再落到地上，又成為一棵棵大樹。這顆種子就是人的一個善心，就是對教育的一個使命，它會產生非常非常大的作用。
王琦老師他就是一個很好的例子、樣板，他一個人把一個班帶好了。呂杰校長也是非常有心的校長，當他經過這個班級，覺得這個班級的孩子非常專注，不會受外面的影響，讓他印象深刻。升旗的時候，看著這班的學生精神抖擻，都不會亂動，非常有威儀。留下的這些印象，讓他開始去了解，原來是王琦老師把中華傳統文化這些倫理道德的教育、孝道的教育、德行的教育融入了他的班級。他非常歡喜感動，就從這一個班，在他全校推展。
由於他全校推展，德行提升得很快。剛好那一年高考在他們學校，學校一進門就有一個孔子像。因為學校是考場，非常多的學校的學生都來了，不只是松花江的學生。可是只要是松花江的學生，都會恭恭敬敬的到孔子像面前鞠躬才進去。結果考試的教育官員，他就覺得怎麼有這些學生這麼恭敬給夫子像行禮？覺得是一道很美麗的風景線。後來一了解，全部都是松花江中學的孩子，所以教育界也開始重視他們學校。而那一年考出來的成績，他們本來優秀成績，差不多是另一個學校的一半，就是考得好的學生，本來是他們只有他們的一半。結果那一年考出來，他們學校是那個學校的兩倍。所以成績也提升得非常快。所以整個東北的教育界，也受到松花江中學的影響，他們稱為「松花江中學效應」。
大家想一想，一個人能產生多大的作用？一個班能產生多大的作用？一個學校能產生多大的作用？願有多大，力量就有多大。所以我們老祖先教給我們的教誨，是人生的真理，每一個人這一生只要用心去感悟、去力行，每一個人都可以印證這些真理。就像《弟子規》最後一句話告訴我們的，「勿自暴，勿自棄，聖與賢，可馴致」。這個「致」就是我們這一生只要肯用心去做，就可以契入聖賢的境界，「可馴致」。
《大學》開頭就告訴我們，「古之欲明明德於天下者」，這是胸懷天下的一種人生價值觀。「先治其國；欲治其國者，先齊其家」，我們現在有自己的家庭，還有我們學校這個大家庭。「欲齊其家者，先修其身；欲修其身者，先正其心」，心又是修身的根本。「欲修其身者，先正其心；欲正其心者，先誠其意；欲誠其意者，先致其知；致知在格物」。所以根本在哪？格物。物格而後知至，而後意誠、心正、身修、家齊、國治、天下平。而「格物」就是什麼？格除習性，格除物欲。人的煩惱愈輕，智慧就愈增長。
現在的人物欲太重，「欲令智迷，利令智昏」。做的都是急功近利，甚至是刀頭舐蜜。一支刀前面有一滴蜂蜜，寧可去嘗個甜頭，也顧不到可能舌頭會被切斷。其實人類現在何嘗不是如此？短短的幾十年，已經讓一個美麗的地球，變成很多地方沒有辦法居住。尤其在江浙一帶，我們俗話講的，「上有天堂，下有蘇杭」。江浙一帶多少非常美麗的地方，我們回到小時候，水溝清澈，魚都在那裡游。現在呢？污染非常非常的快。科學家已經提出來，很多地區不能住人，甚至很多地區，本來是青山綠水的地方，污染到成為癌症的高危險區。那不是讓後代子孫連生活的地方都沒有了嗎？那這何嘗不是刀頭舐蜜？就為了嘗點甜頭，把舌頭給割了。就為了嘗點甜頭，急功近利把後代子孫都忽略掉了，都毀掉了。
現在不是哪一個地區急功近利，現在是全世界都急功近利，被功利主義所污染。所以全世界都發生了同樣的問題：環境污染；再來，青少年犯罪率，這是全世界的問題，這提醒了人類，沒有好的下一代，人類還有未來嗎？可是人卻為了賺更多的錢，把人生最重要的教育下一代給忘記了。重利，輕義啊！老祖宗告訴我們，「不孝有三，無後為大」，所以人類現在不能只考慮眼前。
我們為人父母、為人老師，這兩個身分，是天地間最尊貴的身分，我們對於孩子的人生影響最大。大家回想一下，一個小孩生出來，會講的第一句話是什麼？女士都說「媽媽」，男士都說「爸爸」。夫妻可不能因為這樣吵架。「先學爸爸」、「先學媽媽」，就吵了，這不行。夫妻和樂，對孩子一生人格的健康是有決定性的影響。
從孩子牙牙學語，他剛會講話，講得很順，整句話可以講，請問十句裡面有幾句是「我爸爸說」、「我媽媽說」？不少句吧？我看超過一半。所以每一對父母都曾經是自己小孩的偶像，是不是？你們的孩子都是你們的粉絲。這叫天性。父母跟子女之間的親愛是天性。教育第一個最可貴的目標，就是讓「父子有親」這個孝道終身保持，這個人是天地之間最幸福的。這種感受，只有自己去做了才知道。當我們雙膝跪下，想著父母一生養育、教育的恩德，給父母磕頭。磕了三個頭，起來了，終於體會到，跪著磕頭比站著舒服太多了。那种跟自己內心性德的相應處，讓自己覺得心裡非常的喜悅，表達了我們對父母這種感恩之心。有幫父母洗過腳的人跟沒有洗過腳的是決定不一樣的。「學而時習之，不亦說乎」，做了的那種喜悅，自己一定可以感覺得到。
我聽「天下父母」節目當中翟俊杰翟導演，他講的一句話讓人很感動。人家問他，他覺得人生最幸福的是什麼？他六十多歲了，他說：「我人生最幸福就是回到我的家，打開門，我能喊一聲娘。」大家聽到一個六十幾歲的老人說這樣的話，你們的腦海裡有沒有浮現什麼影像？六十幾歲的老人跟一兩歲走路還不大穩的孩子有沒有相應處？你有沒有看過一兩歲走路還不穩的孩子，突然看到爸爸媽媽，那個臉笑開花了，是吧？然後這樣邊走邊搖，「媽」，跑過去抱在媽媽的腿上、爸爸的腿上，好像他就是天底下最幸福的人，是吧？請問他有沒有錢？他有沒有地位？可是他是最快樂的人。可是我們現在有錢也有地位，笑得都沒有他燦爛，是不是？對呀，那值得我們思考很多問題。
依照科學家的實驗結果，一個人一兩歲的時候，平均一天笑一百八十次。這個數字我認同，為什麼？剛好我的同事生了一個孩子，四個月，跟我是鄰居。首先第一個重點，他胎教做得不錯，他母親常常讀中華傳統文化的經典給他聽，所以不會吵。有時候會稍微鬧，讀經典給他聽他就不吵，很有效。再來，我常常都會進去看他，他的小名叫恕兒，「忠恕之道」，讓他終身奉行。您看有文化涵養的父母，他就知道給孩子的名字，包含給他人生的每一個引導，都要跟經典相應，利益他一生正確的思想觀念。我常去他們家裡看這個孩子，這個孩子平常沒有人跟他講話，他也會自己笑，呵呵呵就笑了。所以確確實實可以笑一百八十次。

In [ ]:
Markdown(results[0])

Okay, here's the English translation of the provided text, aiming for accuracy and a natural reading flow, while retaining the original tone:

**Editor's Preface**

"Hoping one's son becomes a dragon, and one's daughter a phoenix," is the aspiration of every parent. "To discover talent for the world" is the ambition of every educator. How can we become the guiding light in the lives of our own children, our students, and even everyone around us, using the lamp of love and wisdom to illuminate each other’s path?

In July 2013, the China Teacher Development Foundation and the National Teacher Research Foundation, under the "Twelfth Five-Year Plan" key project, the “Action Research on Chinese School Legal Education,” organized a principal training class in Nanjing with the theme "Learning Traditional Culture to Become Moral Educators." Mr. Cai Lixu was invited to give a free lecture on the topic "Being a Guiding Light in a Child’s Life," which consisted of six class sessions. As the lectures in Nanjing were not yet complete, Mr. Cai continued to record this course every Saturday after returning to the Malaysian Chinese Cultural Education Center, adding another seventeen class sessions. Therefore, the total series of lectures on this topic comprised twenty-three sessions.

With a sincere and selfless heart, Mr. Cai has shared his profound understanding of the five thousand years of Chinese educational wisdom, as well as his over ten years of experience in promoting culture and nurturing talent, generously offering it to the public. This series of lectures is another brilliant presentation by Mr. Cai, following his "Happy Life Lecture" and "Detailed Explanation of the *Di Zi Gui*." It is not only worth careful contemplation by all parents and educators, but also worthy of reference for entrepreneurs and team leaders who strive for the happiness of their employees and the inspiration of the public.

In light of this, our lecture notes team has slightly reorganized Mr. Cai's lecture transcripts for the benefit of our readers. We kindly ask that you, our esteemed readers, not hesitate to offer your valuable guidance if you find any improper compilation or textual errors.

Lecture Notes and Publications Team
Malaysian Chinese Cultural Education Center
July 1, 2014

**Table of Contents**

Outline for "Being a Guiding Light in a Child's Life"

Lecture 1: Explanation of the Topic
Lecture 2: Explanation of the Topic
Lecture 3: The Meaning of Life
Lecture 4: Human Suffering
Lecture 5: Establishing the Nation and Governing the People; Education First
Lecture 6: Establishing the Nation and Governing the People; Education First
Lecture 7: The Dignity of the Teaching Profession: I. The Importance of Teachers
Lecture 8: The Dignity of the Teaching Profession: II. Honoring Teachers Leads to Valuing the Way; III. Revitalizing the Teaching Profession
Lecture 9: The Dignity of the Teaching Profession: III. Revitalizing the Teaching Profession
Lecture 10: The Dignity of the Teaching Profession: III. Revitalizing the Teaching Profession; Role Model for Educators: The Supreme Sage Teacher: I. Confucius’s Approach to Learning
Lecture 11: Role Model for Educators: The Supreme Sage Teacher: II. Confucius’s Conduct as a Teacher
Lecture 12: Role Model for Educators: The Supreme Sage Teacher: II. Confucius’s Conduct as a Teacher
Lecture 13: The Importance of Self-Education: I. Educators Must First Educate Themselves
Lecture 14: The Importance of Self-Education: I. Educators Must First Educate Themselves; II. Courage to Correct One's Mistakes; III. Strict with Self, Lenient with Others
Lecture 15: The Importance of Self-Education: III. Strict with Self, Lenient with Others
Lecture 16: The Importance of Self-Education: III. Strict with Self, Lenient with Others
Lecture 17: The Importance of Self-Education: III. Strict with Self, Lenient with Others
Lecture 18: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
Lecture 19: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
Lecture 20: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
Lecture 21: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
Lecture 22: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
Lecture 23: The Importance of Self-Education: IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity

Outline for “Being a Guiding Light in a Child’s Life”

I. Explanation of the Topic
II. The Meaning of Life
III. Human Suffering
IV. Establishing the Nation and Governing the People; Education First
V. The Dignity of the Teaching Profession
   I. The Importance of Teachers
      1. Without understanding ritual and righteousness, one cannot practice law. (*Qun Shu Zhi Yao 360*, sentence 218)
      2. Jade must be cut and polished to become a useful object.
      3. Ancient scholars compared things to classify them. (*Book of Rites, Record of Learning*)
      4. O Teacher, O Teacher, you hold the fate of the child.
      5. Cultural Transmission
   II. Honoring Teachers Leads to Valuing the Way
      1. In the way of learning, a strict teacher is difficult to find. (*Book of Rites, Record of Learning*)
      2. Therefore, ancient sage kings invariably honored their teachers. When teachers are honored, rank, wealth, and poverty are not considered. (*Qun Shu Zhi Yao 360*, sentence 250)
      3. One part sincerity and respect yields one part benefit; ten parts sincerity and respect yield ten parts benefit. There are top-scoring students, but not top-scoring teachers.
      4. Filial piety to parents is the foundation for respecting teachers.
      5. Contrast between the past and present.
   III. Revitalizing the Teaching Profession
      1. Only by respecting oneself will others respect you.
      2. Truly understanding what it means to be someone's child. (*Qun Shu Zhi Yao 360*, sentence 177)
      3. Understanding one's duty – imparting the Way, teaching skills, and resolving doubts.
      4. Content with poverty, joyful in the Way, cultivating talent for the nation.
      5. Self-improvement.
VI. Role Model for Educators: The Supreme Sage Teacher
   I. Confucius’s Approach to Learning
      1. Relaying, not creating; believing in and loving the ancient. (*Analects, Shu Er, Book VII*)
      2. The Master said, “At fifteen, I set my heart on learning.” (*Analects, Wei Zheng, Book II*)
      3. Even in a village of ten households. (*Analects, Gong Ye Chang, Book V*)
      4. The Master said, “Those who do not ask, ‘What shall I do?’” (*Analects, Wei Ling Gong, Book XV*)
          Knowing it is not as good as loving it. (*Analects, Yong Ye, Book VI*)
      5. The Master said to You, "Shall I teach you what it means to know?" (*Analects, Wei Zheng, Book II*)
      6. A noble person does not seek to fill his belly. (*Analects, Xue Er, Book I*) A scholar should set his mind on the Way. (*Analects, Li Ren, Book IV*)
      7. If one does not cultivate virtue. (*Analects, Shu Er, Book VII*)
      8. Learning without thinking is confusion. (*Analects, Wei Zheng, Book II*)
      9. The Master said, “When walking in a party of three, there must be one I can learn from.” (*Analects, Shu Er, Book VII*)
      10. Si, do you think I am someone who has learned much and remembers it? (*Analects, Wei Ling Gong, Book XV*)
   II. Confucius’s Conduct as a Teacher
      1. Wisdom
           1. Focus on the root.
           2. Possess far-sighted vision.
           3. Forbid things before they happen.
           4. Teach according to individual aptitude.
           5. Seize opportunities for education.
           6. Learn from one another.
      2. Benevolence and Virtue
           1. Lead by example.
           2. Have a compassionate heart.
           3. Teach all without discrimination.
      3. Courage
      4. Conduct
VII. The Importance of Self-Education
   I. Educators Must First Educate Themselves
      1. A noble person works on the root, and when the root is established, the Way will arise.
      2. To dissolve personal biases in temperament is a great learning; to resolve conflicts within the family is a great undertaking.
      3. Don't deceive yourself in your thoughts and actions.
      4. Let go of opposition, control, and possession.
   II. Courage to Correct One's Mistakes
      1. A sense of shame.
      2. A sense of fear.
      3. Courageous heart.
   III. Strict with Self, Lenient with Others
      1. Strict with Self
           1. Do not be greedy.
           2. Do not be angry.
           3. Do not be foolish.
           4. Do not be arrogant.
           5. Do not be doubtful.
      2. Lenient with Others
           1. Put yourself in their place.
           2. Do not be calculative.
           3. Be kind.
   IV. Fulfilling Moral Obligations, Restraining Evil, Preserving Sincerity
      1. Act as a ruler, act as a parent, act as a teacher.
            (1) Ruler: lead by example, sense of mission, selfless, know how to identify talent, accept advice.
            (2) Parent: nurture, have a high degree of trust, do not give up, do not seek reward.
            (3) Teacher: transmit the Way, cultivate pillars of the nation, awaken wisdom.
      2. The Duty of a Minister
           (1) Four types of initiative.
           (2) Focus on the root.
           (3) Uphold integrity, be loyal, offer admonishments, and recommend the virtuous.
      3. Correct Thoughts and Concepts
           (1) Towards people.
           (2) Towards matters.
           (3) Towards things.

**Lecture 1: Explanation of the Topic**

Respected Mr. and Mrs. Ye, Secretary-General Mo, Teacher Chen, fellow educators, esteemed elders, brothers and sisters, good morning to you all!

It is a great joy to see all of you principals and fellow educators here. Confucius said, "It is a pleasure to have friends coming from afar." The backdrop on our stage is a field of rapeseed flowers. The countless beautiful blooms remind us that all of you principals and educators are diligently working and thinking for the happiness of children's lives. Your dedication will surely blossom in the lives of the children, like this rapeseed field. Behind each of you principals, you may be influencing the happiness of tens of thousands, or even hundreds of thousands of children's lives. Therefore, the topic I will be discussing with you today is "Being a Guiding Light in a Child's Life."

Since arriving at the Nanjing Ju Meixin Education Center yesterday, I have been continually moved. Last night, during everyone's self-introductions, a common saying came to mind: "To have one true friend in life is enough to die without regret." Listening to the principals share, I felt that here, we have met many kindred spirits; there is a sense of deep understanding and appreciation for one another. I also experienced what the classics tell us: "Virtue is never alone, it is bound to have neighbors" and "all within the four seas are brothers." Furthermore, today's gathering is an opportunity for us to form a teaching team on the path of promoting sage education. Over a hundred of us will join hands and work together. We can share our good experiences and examples with each other, and that power will be truly immense.

In the past, we fought alone. Now we have met all of these like-minded people from across our motherland, these principals who have a deep sense of mission for education. It is a truly extraordinary gathering. As the saying goes, "understand the affinity, cherish the affinity, create the affinity," and this will create happiness for the teachers, colleagues, and children in all of our schools. We also have the principle of "know gratitude, feel gratitude, repay gratitude." We are grateful to the Nanjing Ju Meixin Education Center for its commitment to promoting sage education and the future of education. They have a deep sense of mission and have put in so much effort to organize this course during the summer holiday. We also appreciate the organizers, the China Teacher Development Foundation under the Ministry of Education, the National Teacher Research Foundation, and this project team, who have worked together with the Ju Meixin Center to bring about this gathering. Behind it all are these dedicated people, with their great care and consideration. In addition, the entire Ju Meixin staff takes good care of us, ensuring we eat and sleep well. We are truly grateful for everything.

Therefore, knowing the gratitude, knowing the effort made by so many people, whenever we meet a staff member, we should show a bright smile to thank them for their contribution. And not just here; back in our schools, let’s thank all the teachers and colleagues for working together, and thank all the parents for their support of the school. We are all living in a world of gratitude.

In fact, as long as a person has a grateful heart, they are bound to be happy. Knowing gratitude, feeling gratitude, what follows is? Repaying gratitude. Through our own self-improvement, and by putting it into practice in our schools, we can truly establish and promote ethics and moral education, making our schools models of schools implementing ethics and moral education in China. This is called illuminating the vast land of China. We already have role models, like Teacher Wang Qi from Songhua River Middle School in Jilin. She devotedly worked in her class, neither seeking fame nor gain, simply hoping to give her children a good future. Just imagine, how much power can one person's sincerity unleash? It might be hard to imagine. It’s like a seed falling into the soil. It roots, it sprouts, it grows strong, and then bears much fruit. And in these fruits, there are more seeds. One single seed can eventually produce many fruits and many seeds, which fall onto the ground and grow into more big trees. That seed is a person's good heart, a mission for education. It can create immense power.

Teacher Wang Qi is a great example, a role model. She single-handedly managed her class extremely well. Principal Lu Jie is also a very dedicated principal. When he passed by her class, he noticed how focused the children were and how unaffected they were by external distractions. It left a deep impression on him. During the flag-raising ceremony, he saw that her students were vibrant and orderly, without any restless movements, and had a commanding presence. These impressions led him to investigate. He discovered that Teacher Wang Qi had incorporated traditional Chinese culture, including ethics and moral education, filial piety, and character development, into her teaching. He was overjoyed and moved, and decided to promote this from one class to the entire school.

Because the school implemented the initiative school-wide, character improved very quickly. By chance, that year's national college entrance exam was held at their school. Upon entering the school gate, there was a statue of Confucius. As it was an exam site, many students from different schools came, not just those from Songhua River Middle School. But all the students from Songhua River Middle School respectfully bowed to the statue of Confucius before entering. The education officials in charge of the exam were surprised to see these students bowing so reverently to the statue of Confucius. They thought it was a beautiful sight. After making inquiries, they discovered that all these students were from Songhua River Middle School, and that's how their school began to gain attention from the educational community. That year, the school's results on the college entrance exam dramatically improved. Their usual percentage of high scoring students had been about half of that of another school. But that year, they surpassed that other school by a factor of two. Thus their academic performance rapidly improved. And the entire educational community in Northeast China was influenced by Songhua River Middle School. It became known as the "Songhua River Middle School Effect."

Just think, how much impact can one person create? How much impact can one class create? How much impact can one school create? The greater the aspiration, the greater the power. Our ancestors taught us principles that are the truth of life. Every person can verify these truths as long as they truly contemplate and practice them. Just as the last sentence in the *Di Zi Gui* tells us, “Do not give up on yourself, do not abandon yourself. With diligence, one can reach the level of a sage or a worthy.” The word "reach" means that as long as we are willing to put our heart into it, we can achieve the realm of sages and worthies.

The opening lines of *The Great Learning* tells us, "In ancient times, those who wished to illuminate their luminous virtue throughout the world," which demonstrates a world-embracing perspective on the values of life. "First governed their nation; those who wished to govern their nation, first ordered their families." We now have our own families, and our schools, which are also great families. "Those who wished to order their families, first cultivated their personal lives; those who wished to cultivate their personal lives, first rectified their hearts," and the heart is the basis of personal cultivation. “Those who wished to rectify their hearts, first made their intentions sincere; those who wished to make their intentions sincere, first extended their knowledge; the extension of knowledge lies in the investigation of things." So, where does the foundation lie? Investigating things. When things are investigated, then knowledge is extended, then intentions are made sincere, then the heart is rectified, then personal life is cultivated, then the family is ordered, then the nation is governed, and then the world is at peace. And the "investigation of things" means what? It means eliminating bad habits and eliminating material desires. The lighter a person's troubles are, the greater their wisdom will grow.

People today have too many material desires; "desire obscures wisdom, profit makes one lose their senses." They pursue quick results and immediate benefits. They are like tasting honey on the edge of a knife. They would rather take a taste of sweetness, neglecting the fact that their tongue might be cut off. In fact, isn’t this what humanity is doing now? In just a few short decades, they have turned a beautiful planet into many places that are uninhabitable. Especially in the Jiangsu and Zhejiang regions, as the saying goes, "In heaven, there is paradise, and on Earth, there is Suzhou and Hangzhou." How many beautiful places were there in the Jiangsu and Zhejiang area. When we were children, the ditches were clear, and fish were swimming there. And now? The pollution is rapid and severe. Scientists have already pointed out that many regions are uninhabitable, and some areas, which used to be beautiful mountains and clear waters, have been polluted into high-risk areas for cancer. Isn't this depriving future generations of their place to live? Isn’t this like tasting honey on the edge of a knife? Just for a little bit of sweetness, the tongue is cut off. Just for a little bit of sweetness, immediate gain takes priority, neglecting and destroying future generations.

Now, it’s not just one area that is pursuing quick results; it’s the entire world that is pursuing quick results and is being polluted by utilitarianism. That's why the whole world is facing similar problems: environmental pollution, and also rising juvenile crime rates. This is a problem worldwide, reminding humanity that without a good next generation, will humanity even have a future? Yet people are pursuing money and have forgotten about the most important thing in life: educating the next generation. They value profit over righteousness! Our ancestors taught us that, "Among the three acts of unfilial piety, not having descendants is the greatest." Humanity cannot just think about the present moment.

As parents and as teachers, our two roles are the most honorable on Earth. We have the greatest impact on our children's lives. Think back, what is the first word a child learns to say? Women will say "Mama," and men will say "Papa." Husbands and wives should not fight about this. “Learning 'Papa' first,” or “learning ‘Mama’ first,” arguing over this is not okay. Harmony between husband and wife has a decisive influence on a child’s lifelong healthy personality.

From the time a child starts babbling, when they can finally speak fluently and can speak whole sentences, out of ten sentences, how many are “my dad said,” or “my mom said?” Quite a few, right? I think more than half. So, every parent was once the idol of their child, right? Your children are your fans. This is innate. The love between parents and children is innate. The most precious goal of education is to maintain the filial love between parent and child for the rest of their lives. This person is the happiest in the world. This feeling can only be known through experience. When we kneel down, thinking of our parents’ lifelong care and education, and kowtow to them, having kowtowed three times, we can finally understand that kneeling and kowtowing is so much more comfortable than standing. That connection to the innate virtue within brings great joy and expresses our gratitude towards our parents. The feeling of having washed a parent’s feet is definitely different from that of not having done so. “To learn something and practice it regularly, isn’t it a joy?” The joy of putting something into practice can definitely be felt.

I was very touched by a statement made by director Zhai Junjie on the “Parents All Over the World” program. Someone asked him what he felt was the most happiness in life, and he, being over sixty years of age, said, "The greatest happiness in my life is returning to my home, opening the door, and being able to call out 'Mother'." When hearing a man in his sixties saying such words, do you visualize anything? Is there a similarity between a man in his sixties and a toddler who can’t even walk steadily? Have you seen toddlers, who can't yet walk very steadily, suddenly see their parents and burst into a smile? They teeter as they walk, calling “Mama,” and run to hug their mother’s or father's legs, as if they are the happiest people on Earth, right? Do they have money? Do they have status? But they are the happiest people. Yet, we, who have money and status, don't smile as brightly as they do, right? Yes, this makes us think a lot.

According to scientific experiments, one to two-year-olds smile an average of one hundred and eighty times a day. I believe this number is correct. Why? Just recently my colleague had a child, who is four months old, and we are neighbors. The first key point is that he did a good job with prenatal education. His mother often read classical traditional Chinese texts to him, so he doesn’t cry. Sometimes he fusses a little, but as soon as she reads the classics to him, he stops crying. It’s very effective. Furthermore, I often go visit him. His nickname is Shuer, meaning "the way of loyalty and forgiveness," to encourage him to practice it throughout his life. You see, parents with a cultural foundation know that they should give their child a name that reflects the classics. They should guide his life with teachings that align with the classics, benefiting his entire life with correct thinking. I often visit this child at their home. Even when no one is talking to him, he smiles to himself. He just chuckles. So it’s definitely true that they can smile one hundred and eighty times.

That's the translation. I hope this is helpful! Please let me know if you have any other requests.


In [ ]:
translated_texts = [result for result in results]

# join the results together by double whitespace
translated_book = "\n\n".join(translated_texts)

# save the translated book
with open(f"/content/translated_book.txt", "w") as f:
  f.write(translated_book)

## Next steps

Now you know how to properly prepare data, check token limitations and translate each chunk to obtain your results. Feel free to try this process with a book that interests you!